### **Bibliotecas**

In [ ]:
from os import mkdir

import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support as score

import warnings

warnings.filterwarnings('ignore')

### **Gerando os embeddings gerais com hmrm**

#### alabama

In [ ]:
# alabama
_ = embeddings_job('alabama', path_alabama, weight=0.1, K=7, embedding_size=50)

#### arizona

In [ ]:
# arizona
_ = embeddings_job('arizona', path_arizona, weight=0.1, K=7, embedding_size=50)

#### virginia

In [ ]:
# viriginia
_ = embeddings_job('virginia', path_virginia, weight=0.1, K=7, embedding_size=50)

#### chicago

In [ ]:
# chicago
_ = embeddings_job('chicago', path_chicago, weight=0.1, K=7, embedding_size=50)

#### georgia

In [ ]:
# georgia
_ = embeddings_job('georgia', path_georgia, weight=0.1, K=7, embedding_size=50)

In [ ]:
_ = embeddings_job('texas', path_texas, weight=0.1, K=7, embedding_size=50)

### **SVM**

In [ ]:
# validação cruzada k-fold no modelo
kf = KFold(n_splits=5, shuffle=True)
split = kf.split(features_alabama.iloc[:, 0:99], features_alabama.iloc[:, 100])
fscores, precisions, recalls = [], [], []

acc = []
precision = []
recall = []
w_avg_f = []
m_avg_f = []

w_avg_p = []
m_avg_p = []

w_avg_r = []
m_avg_r = []

for train_index, test_index in split:
    X_train, Y_train = features_alabama.loc[train_index].iloc[:,
                       :99], features_alabama.loc[train_index].iloc[:, 100]
    x_test, y_test = features_alabama.loc[test_index].iloc[:,
                     :99], features_alabama.loc[test_index].iloc[:, 100]

    model = svm.SVC(
        kernel="linear", decision_function_shape='ovo', class_weight="balanced")
    model.fit(X_train, Y_train)

    y_predicted = model.predict(x_test)

    precision, recall, fscore, support = score(y_test, y_predicted)
    acc.append(accuracy_score(y_test, y_predicted))

    fscores.append(fscore)
    precisions.append(precision)
    recalls.append(recall)

    w_avg_f.append(f1_score(y_test, y_predicted, average='weighted'))
    m_avg_f.append(f1_score(y_test, y_predicted, average='macro'))

    w_avg_p.append(precision_score(
        y_test, y_predicted, average='weighted'))
    m_avg_p.append(precision_score(y_test, y_predicted, average='macro'))

    w_avg_r.append(recall_score(y_test, y_predicted, average='weighted'))
    m_avg_r.append(recall_score(y_test, y_predicted, average='macro'))
    class_labels = sorted(set(y_test))

In [ ]:
name_columns = [x for x in class_labels]
metrics_f = pd.DataFrame(fscores, columns=name_columns)
metrics_p = pd.DataFrame(precisions, columns=name_columns)
metrics_r = pd.DataFrame(recalls, columns=name_columns)

metrics_f["accuracy"] = acc
metrics_f["macro avg"] = m_avg_f
metrics_f["weighted avg"] = w_avg_f

metrics_p["weighted avg"] = w_avg_p
metrics_p["macro avg"] = m_avg_p

metrics_r["weighted avg"] = w_avg_r
metrics_r["macro avg"] = m_avg_r

print("\nMétricas precision:")
display(metrics_p)

print("\n\nMétricas recall:")
display(metrics_r)

print("\n\nMétricas fscore:")
display(metrics_f)

In [ ]:
melted_metrics_f = metrics_f[[0, 1, 2, 3, 4, 5, 6]].melt()
palette = sns.color_palette("husl", n_colors=len(melted_metrics_f["variable"].unique()))

sns.boxplot(x="variable", y="value", hue="variable", data=melted_metrics_f, palette=palette)
plt.xlabel("Metric")
plt.ylabel("Value")
plt.title("Performance Metrics by Fold")
plt.legend(title="Fold", loc="upper right")
plt.show()

Analisando as métricas, podemos concluir que o desempenho do modelo na classificação dos POIs com base no embedding gerado pelo HMRM não é muito alto. Isso sugere que o embedding pode não capturar todas as características importantes dos dados de check-in do Alabama, levando a um desempenho relativamente baixo na classificação dos POIs.

Porém isso também pode ser por causa da definição dos parâmetros do próprio hmrm, talvez seja bom estudar mais por exemplo o número de componentes latentes (k), peso, tamanho do embedding, etc. **=>** ***se for isso, tenho algumas dúvidas:***

***1. faz sentido testar diferentes valores como no exemplo comentado na main? até achar um que dê resultados melhores?***

***2. ou esses resultados são satisfatórios já que o MTL "aprenderia e melhoraria" as informações?***